In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import pandas as pd
import glob
import math
import time

In [2]:
File_path = '/bettik/bouissob/ISOMIP+/'
Bet_path = '/bettik/bouissob/'

In [3]:
os.listdir(File_path)[0:2]

['Ocean4_COM_MITgcm_JPL.nc', 'Ocean1_TYP_FVCOM.nc']

In [4]:
Data_Name = '*COM_NEMO-CNRS.nc'
paths = glob.glob(File_path + Data_Name)

In [5]:
paths

['/bettik/bouissob/ISOMIP+/Ocean1_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean4_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean3_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean2_COM_NEMO-CNRS.nc']

Dans NN, var en x pour simul y<br>
x : T, S, x, y, z<br>
y : MeltRate<br>
1ere Étape : Preparer les données

In [6]:
def get_depth_temp_sali(df, Data, zs):
    ys, IceDraft = df.ny, df.iceDraft
    N_T, N_S = [], []
    ylast = None
    dataset = Data[['temperatureYZ', 'salinityYZ']].to_dataframe().dropna()
    Index_Y = dataset.index.get_level_values('ny')
    for ind, y in enumerate(ys):
        indz = np.abs(np.array(zs + IceDraft[ind])).argmin()
        if y != ylast:
            Slice = dataset.iloc[Index_Y == y].droplevel('ny')
        Z_N = abs(np.array(Slice.index - indz)).argmin() + Slice.index[0]
        New_T = dataset.temperatureYZ.loc[Z_N, y]
        New_S = dataset.salinityYZ.loc[Z_N, y]
        N_T.append(New_T)
        N_S.append(New_S)
    return np.array(N_T), np.array(N_S)

In [7]:
def New_get_depth_temp_sal(df, Data):
    ys, Icedraft = np.array(df.ny), np.array(df.iceDraft)
    Var = ['temperatureYZ', 'salinityYZ']
    y = xr.DataArray(ys)
    iceZ = xr.DataArray(Icedraft)
    Data = Data.assign_coords({'nz':  - Data.nz})
    Filled = Data[Var].interpolate_na(dim="nz", use_coordinate = True, method = 'nearest', fill_value="extrapolate")
    Cur = Filled[Var].interp(ny = y, nz = iceZ)
    T, S = np.array(Cur.to_array())
    return T, S


In [8]:
Data2 = Data.assign_coords({'nz': - Data.nz})

NameError: name 'Data' is not defined

In [ ]:
Data

In [9]:
for path in paths[0:1]:
    dfT = pd.DataFrame()
    Name = path.replace('.nc', '').replace(File_path, '')
    print(path)
    Data = xr.open_dataset(path)
    Data = Data.assign_coords({'nx': Data.x,
                           'ny': Data.y,
                           'nz': Data.z,
                           'nTime': Data.time})
    T = np.array(Data.nTime.as_numpy())
    Horiz_resol = 2 #km/index
    XstartYZ = int(500 / Horiz_resol)
    YstartXZ = int(40 / Horiz_resol)
    Time = len(T)
    Start = time.perf_counter()
    T = []
    df = []
    for t in range(Time):
        if (t+1)%int(Time/10) == 0:
            print('Starting {} / {}    '.format(t+1, Time))
        df = pd.DataFrame()
        df['meltRate'] = (Data['meltRate'].isel(nTime = t).to_dataframe())['meltRate']
        df['thermalDriving'] = Data['thermalDriving'].isel(nTime = t).to_dataframe()['thermalDriving']
        df['halineDriving'] = Data['halineDriving'].isel(nTime = t).to_dataframe()['halineDriving']
        if Data['iceDraft'].ndim == 2:
            df['iceDraft'] = Data['iceDraft'].to_dataframe()['iceDraft']
        else:
            df['iceDraft'] = Data['iceDraft'].isel(nTime = t).to_dataframe()['iceDraft']
        df['date'] = t
        df.dropna(inplace = True)
        df.reset_index(inplace=True)
        T, S = New_get_depth_temp_sal(df, Data.isel(nTime = t))
        #T, S = get_depth_temp_sali(df, Data.isel(nTime = t), Data.z.to_dataframe())
        df['temperatureYZ'] = T
        df['salinityYZ'] = S
        dfT = pd.concat([dfT, df], ignore_index = True)

    print('Done in {} s'.format(time.perf_counter() - Start))
    d = pd.DataFrame(dfT)
    dfT.dropna(inplace = True)
    dfT.reset_index(drop = True, inplace = True)
    dfT.rename(columns={'nx' :'x', 'ny': 'y'}, inplace=True)
    dfT['x'] = dfT['x'] * Horiz_resol
    dfT['y'] = dfT['y'] * Horiz_resol
    #pd.DataFrame.to_csv(dfT, Bet_path + 'Data/data_{}.csv'.format(Name), index = False)

/bettik/bouissob/ISOMIP+/Ocean1_COM_NEMO-CNRS.nc
Starting 24 / 240    
Starting 48 / 240    
Starting 72 / 240    
Starting 96 / 240    
Starting 120 / 240    
Starting 144 / 240    
Starting 168 / 240    
Starting 192 / 240    
Starting 216 / 240    
Starting 240 / 240    
Done in 30.41291857601027 s


In [13]:
(Data.temperatureYZ).min()

<xarray.DataArray 'temperatureYZ' ()>
array(-2.17729044)

In [15]:
Data.thermalDriving

<xarray.DataArray 'thermalDriving' (nTime: 240, ny: 40, nx: 240)>
[2304000 values with dtype=float32]
Coordinates:
  * nx       (nx) float32 3.21e+05 3.23e+05 3.25e+05 ... 7.97e+05 7.99e+05
  * ny       (ny) float32 1e+03 3e+03 5e+03 7e+03 ... 7.5e+04 7.7e+04 7.9e+04
  * nTime    (nTime) object 0000-01-01 00:00:00 ... 0019-12-01 00:00:00
Attributes:
    description:  thermal driving used in the melt calculation
    units:        deg C